In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_titanic = pd.read_csv("/kaggle/input/titanic/train_and_test2.csv")[[ 'Age', 'Fare', 'Sex', 'sibsp','Parch','Pclass','Embarked','2urvived']]

**Synopsis**
*  Titanic was a British passenger liner that sank in 1912 after colliding with an iceberg. Only 31% of passengers survived in this disaster. The goal of this project is to complete the analysis of what sorts of people were likely to survive.

**Data description:**
1. Survival - Survival (0 = No; 1 = Yes)
2. Pclass - Passenger Class (1 = 1st; 2 = 2nd; 3 = 3rd)
3. Sex - Sex (male = 0 ; female =1)
4. Age - Age
5. Sibsp - Number of Siblings/Spouses Aboard
6. Parch - Number of Parents/Children Aboard
7. Fare - Passenger Fare
8. Embarked - Port of Embarkation 

In [ ]:
df_titanic.isnull().sum()

In [ ]:
df_titanic[(df_titanic['Embarked'].isnull()) ==True]

In [ ]:
df_titanic.dropna(inplace = True)

In [ ]:
def FindUnique (col_list):
    for col in  col_list :
        print("Unique values in :" + col)
        print(df_titanic[col].unique())        
    
        

In [ ]:
FindUnique(['Sex','sibsp','Parch','Pclass','Embarked']) 

In [ ]:
df_titanic['Embarked'] = df_titanic['Embarked'].astype(int)
df_titanic.dtypes

#  **Let's start the data visualization !**

In [ ]:
Survived = df_titanic[df_titanic['2urvived'] == 1]
Not_survived = df_titanic[df_titanic['2urvived'] == 0]
print("Total number of people onboarded the ship : " +str(df_titanic['2urvived'].count()))
print("Only " + str(round((Survived['2urvived'].count()/df_titanic['2urvived'].count())*100)) + "%" + " of people survived")
print(str(round((Not_survived['2urvived'].count()/df_titanic['2urvived'].count())*100)) + "%" + " of people didn't survived")

In [ ]:
fig = plt.figure(figsize = (10,8))
ax= fig.add_subplot(111)
df_titanic[['2urvived']].plot(kind = "hist",ax= ax)
ax.set_xticks([0,1])
ax.set_xticklabels(["Didn't survived","Survived"])
ax.set_ylabel("Number of people",)
ax.legend(["Number of people"])
plt.show()

Lets consider people having age less 30 as Young people. Most of the passengers were younger only.

In [ ]:
str(round((df_titanic[df_titanic['Age']<30]['Age'].count()/df_titanic['2urvived'].count())*100)) + "%"

This number is the percetage of passengers who oboarded the ship alone.Which is very high.

In [ ]:
str(round(df_titanic[(df_titanic['sibsp']!= 1) & (df_titanic['Parch']!= 1)]['2urvived'].count()/df_titanic['2urvived'].count()*100))+"%"

Form the above numbers both young and single people onboarded the ship was high.This concludes that there was high probability that 
Rose and Jack would meet. ha ha ha ha !

Lets see which passengers belonging to which gender survived most !

In [ ]:
fe_per =round((df_titanic[(df_titanic['2urvived']==1) & (df_titanic['Sex'] == 1)]['2urvived'].count()/df_titanic[(df_titanic['Sex'] == 1)]['2urvived'].count())*100)
print(str(fe_per) +"%" +" of the female passengers survived")

In [ ]:
ma_per = round((df_titanic[(df_titanic['2urvived']==1) & (df_titanic['Sex'] == 0)]['2urvived'].count()/df_titanic[(df_titanic['Sex'] == 0)]['2urvived'].count())*100)
print(str(ma_per) +"%" +" of the male passengers survived")

It seems that only a few male passengers survived.

# Let try to fit a logistic regression model on the dataset

In [ ]:
df_titanic.head()

but, first lets perform a basic standardisation on thr Age and Fare varibales.

In [ ]:
df_titanic['Age'] = df_titanic['Age']/df_titanic['Age'].max()

df_titanic['Fare'] = df_titanic['Fare']/df_titanic['Fare'].max()

In [ ]:
df_titanic.head().columns

In [ ]:
Lets transform few variables to categorical variables to removes any bias towards any variable having large values. 

In [ ]:
df_titanic = pd.get_dummies(data = df_titanic,columns =['Sex', 'sibsp', 'Parch', 'Pclass', 'Embarked'])

Lets see if we can fins any strong coorelation between target variable 2urvvied and any dependent variables.

In [ ]:
fig = plt.figure(figsize = (20,8))
ax= fig.add_subplot(111)
sns.heatmap(df_titanic.corr(),ax=ax,annot= True)


In [ ]:
X = df_titanic[['Age', 'Fare',  'Sex_0', 'Sex_1', 'sibsp_0', 'sibsp_1',
       'sibsp_2', 'sibsp_3', 'sibsp_4', 'sibsp_5', 'sibsp_8', 'Parch_0',
       'Parch_1', 'Parch_2', 'Parch_3', 'Parch_4', 'Parch_5', 'Parch_6',
       'Parch_9', 'Pclass_1', 'Pclass_2', 'Pclass_3', 'Embarked_0',
       'Embarked_1', 'Embarked_2']]
y = df_titanic['2urvived']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
clf = LogisticRegression().fit(X_train,y_train)
y_hat = clf.predict(X_test)
f1_score(y_test,y_hat)

In [ ]:
cnf_matrix =confusion_matrix(y_test,y_hat)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
class_names=[0,1] # name  of classes
fig, ax = plt.subplots()
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)
# create heatmap
sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, cmap="YlGnBu" ,fmt='g')
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Confusion matrix', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')

In [ ]:
print("Accuracy:",metrics.accuracy_score(y_test,y_hat))
print("Precision:",metrics.precision_score(y_test,y_hat))
print("Recall:",metrics.recall_score(y_test,y_hat))

**Accurcy of the model is pretty much average only.It is not performing well for prediction of if a surviving of a passenger.
But it doing great on prediction of not survivig of a passenger**

# Thank You For Your Time. 
# If You find my notebook useful please give an UPVOTE.